### Server Setup

Creating a stream of data & sending it to the clients.

<br>
<br>

In [1]:
# to specify delay between two consecutive records in stream
DELAY = 0.1

In [2]:
# Importing required modules

# data streaming
import asyncio
import websockets
from time import sleep

# data processing
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
# Read csv files of merged data

# # original data
# data = pd.read_csv("../data/merged_data.csv", low_memory=False)

# test data for presentation purpose
data = pd.read_csv("../data/test_heterogeneous.csv", low_memory=False)

In [4]:
# dropping NaN or NA values
data = data.dropna()

# encoding string type variables to numeric type (required for model training)
encoder=LabelEncoder()
data['fridge_temp_condition']=encoder.fit_transform(data['fridge_temp_condition'])
data['door_state']=encoder.fit_transform(data['door_state'])
data['door_sphone_signal']=encoder.fit_transform(data['door_sphone_signal'])
data['door_type']=encoder.fit_transform(data['door_type'])
data['ml_light_status']=encoder.fit_transform(data['ml_light_status'])

In [5]:
# data["door_label"][data["door_label"]==1]    # get first record of with a different label class
data.head()

,date-time,fridge_temperature,fridge_temp_condition,door_state,door_sphone_signal,door_type,gps_latitude,gps_longitude,modbus_FC1_Read_Input_Register,modbus_FC2_Read_Discrete_Value,modbus_FC3_Read_Holding_Register,modbus_FC4_Read_Coil,ml_motion_status,ml_light_status,thermostat_current_temperature,thermostat_status,weather_temperature,weather_pressure,weather_humidity
0,31-03-19~12:36:52,13.10,0,0,0,2,0.0,10.0,11287,3537,27844,30527,1,1,25.0,0,31.788508,1.035,32.036579
1,31-03-19~12:36:53,8.65,0,0,0,2,0.0,10.0,11287,3537,27844,30527,0,0,25.0,0,41.630997,1.035,30.886165
2,31-03-19~12:36:54,2.00,1,0,0,2,0.0,10.0,11287,3537,27844,30527,0,0,25.0,0,42.256959,1.035,19.755908
3,31-03-19~12:36:55,4.80,1,0,0,2,0.0,10.0,53287,1463,33518,23014,1,1,25.0,0,49.116581,1.035,78.949621
4,31-03-19~12:36:56,10.70,0,0,0,2,0.0,10.0,53287,1463,33518,23014,1,1,25.0,0,24.017085,1.035,40.001059


In [6]:
# https://websockets.readthedocs.io/en/stable/intro/quickstart.html

async def server(websocket):
    """
    Server function to stream data row-wise in a comma separated string format.
    """

    # convert a row to space(" ") separated string format
    temp = data.to_string(header=False,index=False,
                  index_names=False).split('\n')

    # now convert above row to comma(",") separated string format
    rows = [','.join(ele.split()) for ele in temp]
    
    c=0    # variable to keep track of number of records sent

    # while 'c' is less than & equal to total number of records in dataset
    # this loop is responsible for creating data stream
    while(len(data)>=c):
        # send stringified row to the client
        await websocket.send(rows[c])

        # wait of acknowledgement
        ok = await websocket.recv()
        if(bool(ok)):
            c+=1    # if successfully received then increment 'c' by 1
        
        # add artificial delay to simulate a remote server client setup having communication delays due to network traffic
        sleep(DELAY)

async def main():
    async with websockets.serve(server, "localhost", 8765, ping_interval=None):
        await asyncio.Future()  # run forever

await main()

connection handler failed
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/server.py", line 232, in handler
    await self.ws_handler(self)
  File "/tmp/ipykernel_60879/2254914572.py", line 21, in server
    await websocket.send(rows[c])
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/protocol.py", line 620, in send
    await self.ensure_open()
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/protocol.py", line 930, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedOK: sent 1001 (going away); then received 1001 (going away)


CancelledError: 

<br>
<br>
<center><b>End of File</b></center>